In [1]:
#!/usr/bin/env python
# coding: utf-8

import hashlib
from sage.all import *
from random import randint
import time

P = 2 ** 256 - 2 ** 224 + 2 ** 192 + 2 ** 96 - 1
A = P - 3
B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
ORDER_N = 115792089210356248762697446949407573529996955224135760342422259061068512044369
G_POINTx = 48439561293906451759052585252797914202762949526041747995844080717082404635286
G_POINTy = 36134250956749795798585127919587881956611106672985015071877198253568414405109

ZZ = Integers()
Fp = GF(P)
Fn = GF(ORDER_N)

NISTP256 = EllipticCurve(Fp, [A, B])
NISTP256.set_order(ORDER_N)
G_POINT = NISTP256(G_POINTx, G_POINTy)

PRIVATE_KEY = randint(0, ORDER_N - 1)
PUBLIC_KEY = PRIVATE_KEY * G_POINT

In [2]:
def get_gcd(nb1, nb2):
    """
    Using the Euclidian algorithm, it computes the GCD of 2 numbers
    :param nb1: first number
    :param nb2: second number
    :return: the GCD, x that will be used in the modinv() and y
    """
    s = 0
    x = 1
    t = 1
    y = 0
    r = nb2
    gcd = nb1
    while r != 0:
        quotient = gcd // r
        gcd, r = r, gcd - quotient * r
        x, s = s, x - quotient * s
        y, t = t, y - quotient * t
    return gcd, x, y


def modinv(nb1, nb2):
    """
    Reurns the modular invert
    :param nb1: public key
    :param nb2: phi(n)
    :return: private key
    """
    gcd, x, y = get_gcd(nb1, nb2)
    if x < 0:
        x += nb2
    return x

def hashing(message):
    """
    Hashes a plaintext message and converts it to an integer to compute it later
    :param message: [str] Message to hash
    :return: [int] A hash
    """
    hashed_msg = hashlib.sha256(message.encode()).digest()
    return int.from_bytes(hashed_msg, byteorder='big')

# given message m, return signature (r,s)
def ecdsa_sign(m):
    """
    Generates and ECDSA signature by using an order (which is prime), a point G on the elliptic curve,
    the hash of the message from which I want to sign and the private key.
    In the usual case, a random nonce is generated (k) but for the attack it is determined as k = 10.
    To compute the signature proof, I use the modinv function.
    :param m: [str] Message to sign
    :return: [tuple] Pair of integers for the X of a random point R on the curve and the signature proof
    """
    m_hash = hashing(m)

    # Random nonce
    # k = randint(1, ORDER_N - 1)
    # Fixed nonce for attack
    k = 10

    # Random point on the elliptic curve
    random_point = k * G_POINT

    # Computing the X point of the random point R
    random_point_x = int(random_point[0]) % ORDER_N

    # This Rx needs to be != 0 so if it = 0 then I loop to get a different value by re-computing from the nonce step
    while random_point_x == 0:
        # k = randint(1, ORDER_N - 1)
        # Fixed nonce for attack
        k = 10
        random_point = k * G_POINT
        random_point_x = int(random_point[0]) % ORDER_N

    # Computing the modular invert of the nonce to compute the proof
    k_modinv = modinv(k, ORDER_N)
    signature_proof = k_modinv * (m_hash + random_point_x * PRIVATE_KEY) % ORDER_N

    # If the proof is = 0, then I need to loop to get a proof != 0. I need to re-compute from the nonce step.
    while signature_proof == 0:
        # k = randint(1, ORDER_N - 1)
        # Fixed nonce for attack
        k = 10
        random_point = k * G_POINT
        random_point_x = random_point[0] % ORDER_N
        while random_point_x == 0:
            # k = randint(1, ORDER_N - 1)
            # Fixed nonce for attack
            k = 10
            random_point = k * G_POINT
            random_point_x = random_point[0] % ORDER_N

        k_modinv = modinv(k, ORDER_N)
        signature_proof = k_modinv * (m_hash + random_point_x * PRIVATE_KEY) % ORDER_N

    return random_point_x, signature_proof


# given message m and signature (r,s),
# return True or False
def ecdsa_verify(m, signature):
    """
    Decodes the signature_proof to get the random point R. Then by using its X coordinate, I compare it to the original Rx from the
    signature tuple. If they are the same, I return True. If not, then I return False.
    :param m: [str] Message to verify the signature and ot hash
    :param signature: [tuple] Pair of integers of the X coordinate of a R point of the curve and the signature proof
    :return:
    """
    # I need to hash the message with the same hash algorithm
    m_hash = hashing(m)

    # Computing the modular invert of the signature proof
    signature_proof = modinv(signature[1], ORDER_N)

    # Computing the random point R
    random_point = (m_hash * signature_proof) * G_POINT + (signature[0] * signature_proof) * PUBLIC_KEY

    # Getting its X coordinate
    random_point_x = random_point[0]

    # Comparing with the original Rx
    if random_point_x == signature[0]:
        return True
    else:
        return False

In [3]:
msg = "Hello world"
start = time.time()
sig = ecdsa_sign(msg)
end = time.time()
time_process = end - start

print(f"Signature (r, s): ({sig[0]},{sig[1]})")

if ecdsa_verify(msg, sig):
    print("Signature verification: OK")
else:
    print("Signature verification: NOT OK")

print(f"Processing time: {time_process} second(s)")

Signature (r, s): (93611846365601674425599200647886473617443872040541410036779615417472400060991,14635984284972527979296641933668358930563676001333776470534513693390679448980)
Signature verification: OK
Processing time: 0.0005640983581542969 second(s)


In [4]:
# given two lists [m,r,s], return private key d
def attack(list1, list2):
    """
    With a given message and signature, with a known nonce k = 10, by using the nonce, I compute to find the private key.
    :param list: [list] With a message [str], a X coordinate from a random point [int] and the signature proof [int]
    :return: [int] The private key
    """
    # Getting the list elements
    m0 = list1[0]
    r0 = list1[1]
    s0 = list1[2]
    # Hashing the message with the hash algorithm
    m0_hash = hashing(m0)

    m1 = list2[0]
    r1 = list2[1]
    s1 = list2[2]
    # Hashing the message with the hash algorithm
    m1_hash = hashing(m1)

    # Equations to find the private key and the nonce
    s = s0 - s1
    k = (modinv(s, ORDER_N) * (m0_hash - m1_hash)) % ORDER_N
    private_key = (s0*k - m0_hash) * modinv(r0, ORDER_N) % ORDER_N

    return private_key

In [5]:
list = [msg, sig[0], sig[1]]

msg = "Hello world 2"
sig2 = ecdsa_sign(msg)
list2 = [msg, sig2[0], sig2[1]]

result = attack(list, list2)
if result == PRIVATE_KEY:
    print(f"Private key found : {PRIVATE_KEY}")
else:
    print("Private key is not found...")
    print(f"PK from attack : {result}")
    print(f"PK from origin : {PRIVATE_KEY}")

Private key found : 23447219775107700430454461855360251328785264513269495293711933209522872460794
